# Ex 2

In [ ]:
from functools import reduce
from collections import defaultdict

# Example input
text = "the quick brown fox jumps over the lazy dog the fox was quick"

# Step 1: Map phase - create (word, 1) for each word
words = text.split()
mapped_words = list(map(lambda word: (word, 1), words))


# Step 2: Reduce phase - combine counts for each word
def reducer(acc, pair):
    word, count = pair
    acc[word] += count
    return acc


# Use reduce to aggregate word counts
word_freq = reduce(reducer, mapped_words, defaultdict(int))

# Convert defaultdict to a regular dictionary
word_freq = dict(word_freq)
word_freq

{'the': 3,
 'quick': 2,
 'brown': 1,
 'fox': 2,
 'jumps': 1,
 'over': 1,
 'lazy': 1,
 'dog': 1,
 'was': 1}

# Ex 3

In [ ]:
from functools import reduce
from collections import defaultdict

# Example input documents
documents = {"doc1": "dog cat", "doc2": "dog fox", "doc3": "cat mouse"}

# Step 1: Map phase - Use map to create (word, document_id) for each word in each document
mapped_words = list(
    map(
        lambda doc_id_content: [
            (word, doc_id_content[0]) for word in doc_id_content[1].split()
        ],
        documents.items(),
    )
)
mapped_words = [
    item for sublist in mapped_words for item in sublist
]  # Flatten the list of lists


# Step 2: Reduce phase - Use reduce to group document_ids for each word
def reducer(acc, pair):
    word, doc_id = pair
    acc[word].add(doc_id)
    return acc


# Reduce to aggregate the document lists for each word
inverted_index = reduce(reducer, mapped_words, defaultdict(set))

# Convert sets to lists for better readability
inverted_index = {word: list(docs) for word, docs in inverted_index.items()}

# Display the result
inverted_index

{'dog': ['doc2', 'doc1'],
 'cat': ['doc3', 'doc1'],
 'fox': ['doc2'],
 'mouse': ['doc3']}

# Ex 4

In [11]:
from functools import reduce
from collections import defaultdict

# Function to load a graph from a file where each line contains an edge (x, y)
def load_graph(file_path):
    with open(file_path, 'r') as file:
        edges = [tuple(map(int, line.strip().split())) for line in file]
    return edges

# Example usage
file_path = 'data\eulerGraphs3.txt'
graph_edges = load_graph(file_path)

# Step 1: Map phase - Use map to create a list of vertex appearances
vertex_appearances = list(map(lambda edge: [edge[0], edge[1]], graph_edges))
vertex_appearances = [vertex for sublist in vertex_appearances for vertex in sublist]  # Flatten the list

# Step 2: Reduce phase - Count the occurrences of each vertex (degree count)
def count_vertices(acc, vertex):
    acc[vertex] += 1
    return acc

# Reduce to aggregate the degree counts for each vertex
degree_count = reduce(count_vertices, vertex_appearances, defaultdict(int))

# Step 3: Use reduce to count even and odd degree vertices
def count_even_odd(acc, degree):
    if degree % 2 == 0:
        acc['even'] += 1
    else:
        acc['odd'] += 1
    return acc

# Reduce to count even and odd vertices
even_odd_count = reduce(count_even_odd, degree_count.values(), {'even': 0, 'odd': 0})

# Display the result
even_odd_count


<>:11: SyntaxWarning: invalid escape sequence '\e'
<>:11: SyntaxWarning: invalid escape sequence '\e'
C:\Users\acoma\AppData\Local\Temp\ipykernel_37800\1057440648.py:11: SyntaxWarning: invalid escape sequence '\e'
  file_path = 'data\eulerGraphs3.txt'


{'even': 2, 'odd': 98}

# Ex 5

In [14]:
from functools import reduce

# Step 1: Load the graph from the adjacency list file
def load_adjacency_list(file_path):
    with open(file_path, 'r') as file:
        graph = {}
        for line in file:
            node, neighbors = line.strip().split(' : ')
            graph[node] = set(neighbors.split(','))
    return graph

# Step 2: Use map to generate adjacent vertex pairs and their neighbors
def generate_adjacent_pairs(graph):
    return list(map(lambda node: [(node, neighbor, graph[node], graph[neighbor]) 
                                  for neighbor in graph[node] if node < neighbor], graph))

# Step 3: Use reduce to find common neighbors
def find_common_friends(acc, pair):
    for node1, node2, neighbors1, neighbors2 in pair:
        common_neighbors = neighbors1.intersection(neighbors2)
        if common_neighbors:
            acc.append((node1, node2, common_neighbors))
    return acc

# Step 4: Apply map and reduce
def common_friends(file_path):
    graph = load_adjacency_list(file_path)
    
    # Map phase: Generate adjacent vertex pairs
    adjacent_pairs = generate_adjacent_pairs(graph)
    
    # Reduce phase: Find common friends
    return reduce(find_common_friends, adjacent_pairs, [])

# Example usage (replace with your actual file path)
file_path = 'data/friends.txt'
common_friends_list = common_friends(file_path)

# Output the common friends for each pair of adjacent vertices
for node1, node2, common in common_friends_list:
    print(f"{node1}, {node2} : {', '.join(common)}")


1, 8 : 6, 3, 7
1, 6 : 8
1, 7 : 8, 2
1, 2 : 7, 3
1, 3 : 8, 2
2, 7 : 1, 4
2, 3 : 1
2, 4 : 7
3, 8 : 1
4, 8 : 7
4, 7 : 8, 2
6, 8 : 1
7, 8 : 1, 4


# Ex 6

### Problem Setup:
We have a graph represented as a set of edges, and we want to count the number of triangles in this graph. A **triangle** consists of three vertices that are all pairwise connected.

### Map and Reduce Operations:

- **Map**: For each edge \( (a, b, c) \), we will map it to pairs \( \langle a, b \rangle \), where \( a \) is a vertex, and \( b \) is one of its neighbors. The key is the vertex, and the value is its neighbor set. This ensures that each vertex has a list of its neighbors.
  
  $$ \text{map}(a, b, c) \rightarrow \langle a, b \rangle $$

- **Reduce**: For each vertex \( a \) and its associated list of neighbors \( [b_1, b_2, \dots, b_k] \), we will reduce it by finding all the pairs of neighbors \( (b_i, b_j) \) that are also connected. This checks for the existence of common neighbors and counts a triangle every time two neighbors share an edge with \( a \).

  $$ \text{reduce}(a, [b_1, \dots, b_k]) \rightarrow \left(a, \theta([b_1, \dots, b_k])\right) $$

Here, \( \theta([b_1, \dots, b_k]) \) represents the number of triangles formed by vertex \( a \) and its neighbors. Since each triangle is counted three times (once at each vertex), the final result is divided by 3.

### Conclusion:
- The **map** function organizes the graph by vertices and their neighbors.
- The **reduce** function identifies triangles by checking for common neighbors and counting them.


In [1]:
from functools import reduce

# Step 1: Load the graph from the edge list file
def load_graph(file_path):
    with open(file_path, 'r') as file:
        edges = [tuple(map(int, line.strip().split())) for line in file]
    return edges

# Step 2: Create adjacency list from the edges
def build_adjacency_list(edges):
    adjacency_list = {}
    for x, y in edges:
        adjacency_list.setdefault(x, set()).add(y)
        adjacency_list.setdefault(y, set()).add(x)
    return adjacency_list

# Step 3: Map phase - Generate pairs of vertices and their common neighbors
def generate_triangle_candidates(adjacency_list):
    triangle_candidates = []
    for node in adjacency_list:
        for neighbor in adjacency_list[node]:
            if node < neighbor:  # To avoid duplicate pairs (x, y) and (y, x)
                triangle_candidates.append((node, neighbor, adjacency_list[node], adjacency_list[neighbor]))
    return triangle_candidates

# Step 4: Reduce phase - Count triangles by finding common neighbors
def count_triangles(acc, pair):
    node1, node2, neighbors1, neighbors2 = pair
    common_neighbors = neighbors1.intersection(neighbors2)
    acc += len(common_neighbors)  # Each triangle will be counted 3 times (once per vertex)
    return acc

# Full process to count triangles
def triangle_count(file_path):
    edges = load_graph(file_path)
    adjacency_list = build_adjacency_list(edges)
    
    # Map phase: Generate triangle candidates (pairs with common neighbors)
    triangle_candidates = generate_triangle_candidates(adjacency_list)
    
    # Reduce phase: Count the number of triangles
    total_triangle_count = reduce(count_triangles, triangle_candidates, 0)
    
    # Since each triangle is counted 3 times, divide the result by 3
    return total_triangle_count // 3

# Example usage (replace with your actual file path)
file_path = 'data/roadnet.txt'
total_triangles = triangle_count(file_path)

# Output the total number of triangles
print(f"Total number of triangles: {total_triangles}")


Total number of triangles: 120676
